Generates the levels in the folder `./envs_conifig/*`

The environments in `./envs_config/{test_envs or train_envs}/metadata.csv` are defined following the specifications at https://flatland.aicrowd.com/challenges/flatland3/envconfig.html

Similarly from AMLD 2021 and NeurIPS 2020, we set:
* `max_rails_between_cities = 2`
* `grid_mode = False`
* `malfunction_duration = [20,50]` 
* `malfunction_interval = 250 * (env_id+1)`, i.e. {250, ..., 2500}, just slightly different from the {0, ..., 2250} of AMLD 2021 and NeurIPS 2020

Additionally, we chose:
* `max_rail_pairs_in_city = 2`, to train on the hardest scenario

In [1]:
import os
import pandas as pd
from tqdm import tqdm

from flatland.envs.line_generators import sparse_line_generator
from flatland.envs.malfunction_generators import MalfunctionParameters, ParamMalfunctionGen
from flatland.envs.persistence import RailEnvPersister
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_generators import sparse_rail_generator

from utils.persister import save_env_to_pickle

In [ ]:
eval_list = [
    "n_agents",
    "x_dim",
    "y_dim",
    "n_cities",
    "max_rail_pairs_in_city",
    "n_envs_run",
    "grid_mode",
    "max_rails_between_cities",
    "malfunction_duration_min",
    "malfunction_duration_max",
    "malfunction_interval",
    "speed_ratios",
]

"""
The flatland-evaluator uses a specific format for the environment pickle, so we use RailEnvPersister.save for the test environments.
For the training environments, we use our own save_env_to_pickle function, which e.g. saves also the random_seed.
"""
MODE = "train"  # must be set to either "test" or "train"
if MODE == "test":
    PATH = "./envs_config/test_envs" 
    save_function = lambda env, path: RailEnvPersister.save(env, path, save_distance_maps=True)
elif MODE == "train":
    PATH = "./envs_config/train_envs"
    save_function = lambda env, path: save_env_to_pickle(env, path)
else:
    raise ValueError("Invalid mode")

parameters_flatland = pd.read_csv(PATH + "/metadata.csv", index_col=0)
parameters_flatland[eval_list] = parameters_flatland[eval_list].applymap(
    lambda x: eval(str(x))
)

for idx, env_config in tqdm(
    parameters_flatland.iterrows(), total=parameters_flatland.shape[0]
):
    env_config = env_config.to_dict()
    if not os.path.exists(os.path.join(PATH, env_config["test_id"])):
        os.mkdir(os.path.join(PATH, env_config["test_id"]))


    malfunction_parameters = MalfunctionParameters(
        malfunction_rate=1 / env_config["malfunction_interval"],
        min_duration=env_config["malfunction_duration_min"],
        max_duration=env_config["malfunction_duration_max"],
    )

    env_args = {
        'width': env_config["x_dim"],
        'height': env_config["y_dim"],
        'rail_generator': sparse_rail_generator(
            max_num_cities=env_config["n_cities"],
            grid_mode=env_config["grid_mode"],
            max_rails_between_cities=env_config["max_rails_between_cities"],
            max_rail_pairs_in_city=env_config["max_rail_pairs_in_city"],
        ),
        'line_generator': sparse_line_generator(env_config["speed_ratios"]),
        'number_of_agents': env_config["n_agents"],
        'malfunction_generator': ParamMalfunctionGen(malfunction_parameters),
    }

    if MODE == "train":
        env_args["random_seed"] = env_config["random_seed"]

    env = RailEnv(**env_args)
    env.reset()   # TODO: remove this line(?)
    level_id = env_config["env_id"]
    save_function(env, os.path.join(PATH, env_config["test_id"], f"{level_id}.pkl"))

100%|██████████| 4/4 [00:00<00:00, 1443.82it/s]

{1.0: 0.25, 0.5: 0.25, 0.33: 0.25, 0.25: 0.25}
{1.0: 0.25, 0.5: 0.25, 0.33: 0.25, 0.25: 0.25}
{1.0: 0.25, 0.5: 0.25, 0.33: 0.25, 0.25: 0.25}
{1.0: 0.25, 0.5: 0.25, 0.33: 0.25, 0.25: 0.25}


: 